# Visualizing LFP Responses to Stimulus
A very useful view when working with ecephys data is the **LFP trace**. LFP, or Local Field Potential, is the electrical potential recorded in the extracellular space in brain tissue, and represents activity in regions of neurons. This is particularly useful when you examine LFP responses to stimulus events. The type of stimulus can vary, but in order to visualize this, you must have access to the times of the stimulus events you're interested in. In this notebook, you can extract stimulus times from *spike_times.nwb* and LFP data from *probeA_lfp.nwb*, or a similar file. Importantly, since the stimulus timestamps and the LFP timestamps are not likely to be aligned with each other and in perfectly regular intervals, they must be interpolated.

### Environment Setup

In [ ]:
from pynwb import NWBHDF5IO
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from math import sqrt

### Downloading NWB Files
If you don't already have files to analyze, you can use data from The Allen Institute's `Visual Coding - Neuropixels` dataset. If you want to choose your own files to download, set `dandiset_id`, `dandi_stim_filepath`, `dandi_lfp_filepath` accordingly.

In [ ]:
dandiset_id = "000021"
dandi_stim_filepath = ""
dandi_lfp_filepath = ""
download_loc = "~/data"

In [ ]:
my_dandiset = dandiapi.DandiAPIClient().get_dandiset(dandiset_id)
file = my_dandiset.get_asset_by_path(filepath)
filename = filepath.split("/")[-1]
# this may take awhile, especially if the file to download is large
file.download(f"{download_loc}/{filename}")

print(f"Downloaded file to {download_loc}/{filename}")

### Extracting Stimulus Times
First, you must take the stimulus table from your stimulus file. Since your stimulus table will be unique to your experiment, you'll have to use some ingenuity to extract the timestamps that are of interest to you. Below, we display your stimulus names. Set `stim_name` to be the name that contains the associated stimulus table you want. Then we display the stimulus table. You can see that it contains the `start_time` of each stimulus event. In the commented cell below showing `extract timestamps for given stimulus frame`, you should write code to iterate through this table and filter all but the rows that contain an important stimulus event. The output should be a list of timestamps.

In [ ]:
stim_filepath = f"{download_loc}/{filename}"

In [ ]:
stim_io = NWBHDF5IO(stim_filepath, mode="r", load_namespaces=True)
stim_file = stim_io.read() 
stimulus_names = list(stim_file.intervals.keys())
print(stimulus_names)

In [ ]:
stim_name = "Stimulus Name"
stim_table = stim_file.intervals[stim_name]
print(len(stim_table))

In [ ]:
stim_table[0:100]

In [ ]:
### extract timestamps for given stimulus frame

# def cond(x):
#         return True

# filtered_stim_rows = list(filter(cond, stim_table))
# stim_timestamps = [float(row.start_time) for row in filtered_stim_rows]
# print(stim_timestamps)

In [ ]:
len(stim_timestamps)

### LFP Interpolation
After you have a valid list of stimulus timestamps, you can extract the `LFP.data` and associated `LFP.timestamps`. With these, you can generate a regular timestamp array called `time_axis`, and interpolate the LFP data along it, making interpolated LFP data called `interp_lfp`. This should be a 2D array with dimensions `time` and `channel`, where channels are the different measurement channels along the probe. Here, the timestamps are interpolated to 1000 Hz, but you can change this by setting `interp_hz`.

In [ ]:
lfp_filepath = f"{download_loc}/{filename}"
interp_hz = 1000

In [ ]:
lfp_io = NWBHDF5IO(lfp_filepath, mode="r", load_namespaces=True)
lfp_file = lfp_io.read()
lfp = lfp_file.acquisition["probe_0_lfp_data"]

print(lfp.timestamps.shape)
print(lfp.data.shape)

In [ ]:
# ensure we don't go out of bounds on either timestamps array
stop_time = min(lfp.timestamps[-1], stim_timestamps[-1])
# generate regularly-space x values and interpolate along it
time_axis = np.arange(0, stop_time, step=(1/interp_hz))
f = interpolate.interp1d(lfp.timestamps, lfp.data, axis=0, kind="nearest", fill_value="extrapolate")
interp_lfp = f(time_axis)

print(interp_lfp.shape)

### Getting Stimulus Time Windows
Now that you have your interpolated LFP data, you can use the stimulus times to identify the windows of time in the LFP data that exist around a stimulus event. Set `start_time` to be a negative integer, representing the number of seconds before the stimulus event and `end_time` to be number of milliseconds afterward. Then the `windows` array will be generated as a set of slices of the `interp_lfp` trace by using `interp_hz` to convert seconds to array indices. These will be averaged out for each measurement channel.

In [ ]:
start_time = -20
end_time = 200

In [ ]:
# get event windows

windows = []
for stim_ts in stim_timestamps:
    # convert time to index
    start_idx = int(stim_ts*interp_hz) + start_time
    end_idx = int(stim_ts*interp_hz) + end_time
 
    # bounds checking
    if start_idx < 0 or end_idx > len(intp_lfp)-1:
        continue
        
    windows.append(interp_lfp[start_idx:end_idx])

windows = np.array(windows)
print(windows.shape)

In [ ]:
# get average of all windows

average_trace = np.average(windows, axis=0)
average_trace.shape

In [ ]:
# get standard deviation for confidence interval

n = windows.shape[1]
# ci = np.std(windows, axis=0) / sqrt(n)
ci = np.std(windows, axis=0) / 2
ci.shape

### Visualizing LFP Traces
Now you have the averaged LFP traces for each channel. Below are three views of the same data. There are many channels to view, so for convenience, you can view just a subset of all the channels. You can set `start_channel` and `end_channel` to the bounds of the subset you want to view.

In [ ]:
# number of channels
print(average_trace.shape[1])

In [ ]:
start_channel = 0
end_channel = average_trace.shape[1]
n_channels = end_channel - start_channel

#### Traces from Channels Overlaid

In [ ]:
%matplotlib inline

xaxis = np.arange(start_time, end_time)
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(xaxis, average_trace[:,start_channel:end_channel])

plt.show()

#### Traces from Channels Stacked

In [ ]:
%matplotlib inline

xaxis = np.arange(start_time, end_time)
fig, ax = plt.subplots(figsize=(8, n_channels))

for channel in range(start_channel, end_channel):
    offset_idx = channel-start_channel
    offset_trace = average_trace[:,channel] + 0.0001*offset_idx
    ax.plot(xaxis, offset_trace)

plt.show()

#### Traces from Channels with Confidence Intervals

In [ ]:
%matplotlib inline

xaxis = np.arange(start_time, end_time)
fig, axs = plt.subplots(n_channels, 2, figsize=(16, n_channels*2))

for i in range(n_channels):
    for j in range(2):
        channel = start_channel + i

        axs[i][j].plot(xaxis, average_trace[:,channel])
        upper_bound = average_trace + (ci)
        lower_bound = average_trace - (ci)
        axs[i][j].fill_between(xaxis, lower_bound[:,channel], upper_bound[:,channel], color='b', alpha=.1)
